## Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, random_split

## Data Preparation

In [3]:
df=pd.read_csv("cars-85.csv")
df.dropna(inplace=True)
df.shape

(193, 24)

In [4]:
for i in df.columns:
    if df[i].dtypes=="object":
        df[i] = df[i].astype('category')
        df[i] =df[i].cat.codes

In [5]:
targets = df[["price"]].to_numpy()
inputs = df.drop(["price"], axis=1).to_numpy()  

In [6]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
targets = torch.from_numpy(targets)
targets = targets.float()

In [7]:
inputs_mean = torch.mean(inputs, axis=0)
targets_mean = torch.mean(targets, axis=0)
inputs = inputs / inputs_mean  
targets = targets / targets_mean 

In [8]:
dataset = TensorDataset(inputs, targets)

In [9]:
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

## Define Two Models

In [10]:
model1 = nn.Sequential(nn.Linear(23, 16),
                      nn.Sigmoid(),
                       
                      nn.Linear(16, 8),
                      nn.Softmax(),
                       
                       
                      nn.Linear(8, 4),
                      nn.ReLU(),

                      
                      nn.Linear(4, 1),
                      nn.Sigmoid())

model2 = nn.Sequential(nn.Linear(23, 1),
                       nn.ReLU())

In [11]:
# Utility function to train the model
def fit(num_epochs, model, criterion, optimizer, train_loader):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb,yb in train_loader:
            # 1. Generate predictions
            pred = model(xb)
            # 2. Calculate loss
            loss = criterion(pred, yb)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            optimizer.step()
            # 5. Reset the gradients to zero
            optimizer.zero_grad()
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

## Fit Model1

In [62]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model1.parameters(), lr=1e-4)
fit(100, model1, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 0.0001
Epoch [20/100], Loss: 0.0016
Epoch [30/100], Loss: 0.0043
Epoch [40/100], Loss: 0.1583
Epoch [50/100], Loss: 0.0019
Epoch [60/100], Loss: 0.0251
Epoch [70/100], Loss: 0.0006
Epoch [80/100], Loss: 0.1572
Epoch [90/100], Loss: 0.0033
Epoch [100/100], Loss: 0.6456


## Fit Model1 with regularization

In [13]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model1.parameters(),weight_decay=2, lr=1e-4)
fit(100, model1, criterion, optimizer, train_loader)

C:\Users\user\Anaconda3\lib\site-packages\torch\nn\modules\container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [10/100], Loss: 0.5819
Epoch [20/100], Loss: 0.0719
Epoch [30/100], Loss: 0.0029
Epoch [40/100], Loss: 0.2061
Epoch [50/100], Loss: 0.0090
Epoch [60/100], Loss: 5.2673
Epoch [70/100], Loss: 0.1568
Epoch [80/100], Loss: 0.5662
Epoch [90/100], Loss: 0.1686
Epoch [100/100], Loss: 0.6190


## Fit Model2

In [63]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model2.parameters(), lr=1e-4)
fit(100, model2, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 0.4550
Epoch [20/100], Loss: 1.0354
Epoch [30/100], Loss: 0.2205
Epoch [40/100], Loss: 0.6022
Epoch [50/100], Loss: 0.2616
Epoch [60/100], Loss: 0.2735
Epoch [70/100], Loss: 0.1764
Epoch [80/100], Loss: 1.8308
Epoch [90/100], Loss: 0.4550
Epoch [100/100], Loss: 1.8719


## Fit Model2 with Regularization

In [14]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model2.parameters(),weight_decay=2 ,lr=1e-4)
fit(100, model2, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 5.9099
Epoch [20/100], Loss: 0.2735
Epoch [30/100], Loss: 0.1899
Epoch [40/100], Loss: 0.7413
Epoch [50/100], Loss: 0.2944
Epoch [60/100], Loss: 0.0859
Epoch [70/100], Loss: 0.1442
Epoch [80/100], Loss: 0.3545
Epoch [90/100], Loss: 3.0089
Epoch [100/100], Loss: 1.1468
